# A little extra!

## New addition to Week 1

### The Unreasonable Effectiveness of the Agent Loop

# What is an Agent?

## Three competing definitions

1. AI systems that can do work for you independently - Sam Altman

2. A system in which an LLM controls the workflow - Anthropic

3. An LLM agent runs tools in a loop to achieve a goal

## The third one is the new, emerging definition

But what does it mean?

Let's make it real.

In [1]:
# Start with some imports - rich is a library for making formatted text output in the terminal

from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json
load_dotenv(override=True)

True

In [2]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [3]:
openai = OpenAI()

In [4]:
# Some lists!

todos = []
# Truthy/falsy values!
completed = []

In [5]:
def get_todo_report() -> str:
    result = ""
    for index, todo in enumerate(todos):
        if completed[index]:
            result += f"Todo #{index + 1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"Todo #{index + 1}: {todo}\n"
    show(result)
    return result

In [6]:
get_todo_report()

''

In [7]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [8]:
def mark_complete(index: int, completion_notes: str) -> str:
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report()

In [9]:
todos, completed = [], []

create_todos(["Buy groceries", "Finish extra lab", "Eat banana"])

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: Buy groceries\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [10]:
mark_complete(1, "bought")

bought

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: [green][strike]Buy groceries[/strike][/green]\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [11]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
                  
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [12]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [13]:
tools = [{"type": "function", "function": create_todos_json},
        {"type": "function", "function": mark_complete_json}]

In [14]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [15]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-5.2", messages=messages, tools=tools, reasoning_effort="none")
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    show(response.choices[0].message.content)

In [16]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A train leaves Boston at 2:00 pm traveling 60 mph.
Another train leaves New York at 3:00 pm traveling 80 mph toward Boston.
When do they meet?
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [17]:
todos, completed = [], []
loop(messages)

Todo #1: Identify missing quantity (distance between Boston and New York) and choose a reasonable standard 
estimate.
Todo #2: Set up relative motion equation using departure times and speeds.
Todo #3: Solve for meeting time after 2:00 pm, then convert to clock time.
Todo #4: Report result clearly and note dependency on assumed distance.

The problem omits the Boston–New York distance. Use a common textbook estimate of 200 miles between the cities 
(approximate driving distance).

Todo #1: Identify missing quantity (distance between Boston and New York) and choose a reasonable standard 
estimate.
Todo #2: Set up relative motion equation using departure times and speeds.
Todo #3: Solve for meeting time after 2:00 pm, then convert to clock time.
Todo #4: Report result clearly and note dependency on assumed distance.

Let t = hours after 2:00 pm when they meet.
- Boston train travels 60t miles.
- New York train leaves 1 hour later, so it travels 80(t-1) miles (for t ≥ 1).
Meeting condition: 60t + 80(t-1) = D, with D = 200 miles.

Todo #1: Identify missing quantity (distance between Boston and New York) and choose a reasonable standard 
estimate.
Todo #2: Set up relative motion equation using departure times and speeds.
Todo #3: Solve for meeting time after 2:00 pm, then convert to clock time.
Todo #4: Report result clearly and note dependency on assumed distance.

Solve: 60t + 80(t-1) = 200 → 60t + 80t - 80 = 200 → 140t = 280 → t = 2 hours after 2:00 pm.
So they meet at 4:00 pm.

Todo #1: Identify missing quantity (distance between Boston and New York) and choose a reasonable standard 
estimate.
Todo #2: Set up relative motion equation using departure times and speeds.
Todo #3: Solve for meeting time after 2:00 pm, then convert to clock time.
Todo #4: Report result clearly and note dependency on assumed distance.

Provide the meeting time and note it depends on the assumed Boston–NY distance; also give a general formula t = 
(D+80)/140 hours after 2:00 pm.

Todo #1: Identify missing quantity (distance between Boston and New York) and choose a reasonable standard 
estimate.
Todo #2: Set up relative motion equation using departure times and speeds.
Todo #3: Solve for meeting time after 2:00 pm, then convert to clock time.
Todo #4: Report result clearly and note dependency on assumed distance.

Assuming the Boston–New York distance is about 200 miles:

Let t be the number of hours after 2:00 pm when they meet (so t ≥ 1).

- Boston train distance: 60t
- New York train leaves at 3:00 pm, so it travels for t−1 hours: distance 80(t−1)

They meet when the distances add to 200:
60t + 80(t−1) = 200  
140t − 80 = 200 → 140t = 280 → t = 2

So they meet 2 hours after 2:00 pm, i.e. at:

4:00 pm

General form (if the distance is D miles): they meet (D + 80)/140 hours after 2:00 pm.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Now try to build an Agent Loop from scratch yourself!<br/>
            Create a new .ipynb and make one from first principles, referring back to this as needed.<br/>
            It's one of the few times that I recommend typing from scratch - it's a very satisfying result.
            </span>
        </td>
    </tr>
</table>